In [34]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
from tensorflow.keras.applications.resnet_v2 import preprocess_input as preprocess_resnet50v2
tf.get_logger().setLevel(logging.ERROR)

In [35]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [36]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [37]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [38]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [39]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.
Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [40]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [31]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.resnet50_v2.preprocess_input

AttributeError: module 'tensorflow.keras.applications' has no attribute 'resnet50_v2'

In [41]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_resnet50v2(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_resnet50v2(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_resnet50v2(x), y))

In [42]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [43]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.ResNet50V2(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [44]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_2, False
Layer: conv1_pad, False
Layer: conv1_conv, False
Layer: pool1_pad, False
Layer: pool1_pool, False
Layer: conv2_block1_preact_bn, False
Layer: conv2_block1_preact_relu, False
Layer: conv2_block1_1_conv, False
Layer: conv2_block1_1_bn, False
Layer: conv2_block1_1_relu, False
Layer: conv2_block1_2_pad, False
Layer: conv2_block1_2_conv, False
Layer: conv2_block1_2_bn, False
Layer: conv2_block1_2_relu, False
Layer: conv2_block1_0_conv, False
Layer: conv2_block1_3_conv, False
Layer: conv2_block1_out, False
Layer: conv2_block2_preact_bn, False
Layer: conv2_block2_preact_relu, False
Layer: conv2_block2_1_conv, False
Layer: conv2_block2_1_bn, False
Layer: conv2_block2_1_relu, False
Layer: conv2_block2_2_pad, False
Layer: conv2_block2_2_conv, False
Layer: conv2_block2_2_bn, False
Layer: conv2_block2_2_relu, False
Layer: conv2_block2_3_conv, False
Layer: conv2_block2_out, False
Layer: conv2_block3_preact_bn, False
Layer: conv2_block3_preact_relu, False
Layer: conv2_block3_1_

In [45]:
len(base_model.layers)

190

In [46]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [47]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [48]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
#x = aug_layer(inputs)
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'Resnet_50V2')

model_1.summary()

Model: "Resnet_50V2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_10 (Ba  (None, 2048)              8192      
 tchNormalization)                                               
                                                                 
 dense_10 (Dense)            (None, 512)               

In [49]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [50]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [51]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [52]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 10, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/10
73/74 [============================>.] - ETA: 0s - loss: 10.2646 - accuracy: 0.4919
Epoch 1: val_loss improved from inf to 7.64139, saving model to Trained_Models/Resnet_50V2
74/74 [==============================] - 15s 164ms/step - loss: 10.2359 - accuracy: 0.4913 - val_loss: 7.6414 - val_accuracy: 0.7418 - lr: 0.0010
Epoch 2/10
73/74 [============================>.] - ETA: 0s - loss: 6.1003 - accuracy: 0.6331
Epoch 2: val_loss improved from 7.64139 to 5.11862, saving model to Trained_Models/Resnet_50V2
74/74 [==============================] - 13s 174ms/step - loss: 6.0888 - accuracy: 0.6324 - val_loss: 5.1186 - val_accuracy: 0.8170 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - ETA: 0s - loss: 4.1416 - accuracy: 0.6789
Epoch 3: val_loss improved from 5.11862 to 3.52397, saving model to Trained_Models/Resnet_50V2
74/74 [==============================] - 13s 168ms/step - loss: 4.1416 - accuracy: 0.6789 - val_loss: 3.5240 - val_accuracy: 0.8121 - lr: 0.0010


In [53]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 39ms/step - loss: 1.0783 - accuracy: 0.7657


[1.0783250331878662, 0.7656804919242859]

In [54]:
base_model.trainable = True

In [55]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [56]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [57]:
start_epoch = 10
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_50V2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_10 (Ba  (None, 2048)              8192      
 tchNormalization)                                               
                                                                 
 dense_10 (Dense)            (None, 512)               

In [58]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 11/20
74/74 [==============================] - ETA: 0s - loss: 1.7312 - accuracy: 0.5540
Epoch 11: val_loss did not improve from 1.13378
74/74 [==============================] - 9s 78ms/step - loss: 1.7312 - accuracy: 0.5540 - val_loss: 253.9209 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 12/20
73/74 [============================>.] - ETA: 0s - loss: 1.2390 - accuracy: 0.6785
Epoch 12: val_loss did not improve from 1.13378
74/74 [==============================] - 4s 51ms/step - loss: 1.2363 - accuracy: 0.6794 - val_loss: 71.4288 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 13/20
73/74 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.5736
Epoch 13: val_loss did not improve from 1.13378
74/74 [==============================] - 4s 51ms/step - loss: nan - accuracy: 0.5701 - val_loss: nan - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 14/20
73/74 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.3018
Epoch 14: val_loss did not improve from 1.13

KeyboardInterrupt: 

In [35]:
start_epoch = 20
for layer in base_model.layers[-75:]:
    layer.trainable = True

In [36]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_1 (Bat  (None, 512)               20

In [37]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
47/74 [==================>...........] - ETA: 1s - loss: nan - accuracy: 0.2513

KeyboardInterrupt: 

In [30]:
start_epoch = 30
for layer in base_model.layers[-300:]:
    layer.trainable = True

In [31]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00009), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 128)               32896

In [32]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/40
74/74 [==============================] - ETA: 0s - loss: 0.4440 - accuracy: 0.8619
Epoch 31: val_loss did not improve from 0.08031
74/74 [==============================] - 54s 291ms/step - loss: 0.4440 - accuracy: 0.8619 - val_loss: 0.1423 - val_accuracy: 0.9594 - lr: 9.0000e-05
Epoch 32/40
74/74 [==============================] - ETA: 0s - loss: 0.3214 - accuracy: 0.8937
Epoch 32: val_loss did not improve from 0.08031
74/74 [==============================] - 19s 256ms/step - loss: 0.3214 - accuracy: 0.8937 - val_loss: 0.1051 - val_accuracy: 0.9674 - lr: 9.0000e-05
Epoch 33/40
74/74 [==============================] - ETA: 0s - loss: 0.3136 - accuracy: 0.8967
Epoch 33: val_loss did not improve from 0.08031
74/74 [==============================] - 20s 266ms/step - loss: 0.3136 - accuracy: 0.8967 - val_loss: 0.2196 - val_accuracy: 0.9357 - lr: 9.0000e-05
Epoch 34/40
74/74 [==============================] - ETA: 0s - loss: 0.3187 - accuracy: 0.8920
Epoch 34: val_loss did not im

In [33]:
start_epoch = 40
for layer in base_model.layers[-450:]:
    layer.trainable = True

In [34]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00005), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 128)               32896

In [35]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 41/50
74/74 [==============================] - ETA: 0s - loss: 0.3006 - accuracy: 0.9000
Epoch 41: val_loss did not improve from 0.05755
74/74 [==============================] - 84s 489ms/step - loss: 0.3006 - accuracy: 0.9000 - val_loss: 0.1595 - val_accuracy: 0.9555 - lr: 5.0000e-05
Epoch 42/50
74/74 [==============================] - ETA: 0s - loss: 0.2307 - accuracy: 0.9238
Epoch 42: val_loss did not improve from 0.05755
74/74 [==============================] - 29s 388ms/step - loss: 0.2307 - accuracy: 0.9238 - val_loss: 0.0795 - val_accuracy: 0.9743 - lr: 5.0000e-05
Epoch 43/50
74/74 [==============================] - ETA: 0s - loss: 0.2274 - accuracy: 0.9327
Epoch 43: val_loss did not improve from 0.05755
74/74 [==============================] - 27s 355ms/step - loss: 0.2274 - accuracy: 0.9327 - val_loss: 0.0609 - val_accuracy: 0.9842 - lr: 5.0000e-05
Epoch 44/50
74/74 [==============================] - ETA: 0s - loss: 0.3318 - accuracy: 0.8916
Epoch 44: val_loss did not im

In [38]:
start_epoch = 50
for layer in base_model.layers[-510:]:
    layer.trainable = True

In [39]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000005), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 128)               32896

In [40]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 51/60
74/74 [==============================] - ETA: 0s - loss: 0.0817 - accuracy: 0.9788
Epoch 51: val_loss did not improve from 0.04952
74/74 [==============================] - 86s 581ms/step - loss: 0.0817 - accuracy: 0.9788 - val_loss: 0.0766 - val_accuracy: 0.9773 - lr: 5.0000e-06
Epoch 52/60
74/74 [==============================] - ETA: 0s - loss: 0.0599 - accuracy: 0.9818
Epoch 52: val_loss did not improve from 0.04952
74/74 [==============================] - 39s 521ms/step - loss: 0.0599 - accuracy: 0.9818 - val_loss: 0.1039 - val_accuracy: 0.9753 - lr: 5.0000e-06
Epoch 53/60
74/74 [==============================] - ETA: 0s - loss: 0.0651 - accuracy: 0.9818
Epoch 53: val_loss did not improve from 0.04952
74/74 [==============================] - 38s 506ms/step - loss: 0.0651 - accuracy: 0.9818 - val_loss: 0.0727 - val_accuracy: 0.9763 - lr: 5.0000e-06
Epoch 54/60
74/74 [==============================] - ETA: 0s - loss: 0.0696 - accuracy: 0.9792
Epoch 54: val_loss did not im

KeyboardInterrupt: 

In [43]:
resnet = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet")


In [46]:
resnet.evaluate(val_datagen)

32/32 [==============================] - 2s 67ms/step - loss: 0.0495 - accuracy: 0.9842


[0.04952496290206909, 0.9841740727424622]